In [1]:
import anndata
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
from dredFISH.Utils.analysisu import *


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
2025-12-08 17:46:29.230742: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.5/lib64:
2025-12-08 17:46:29.230799: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import sys
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")
if current_dir not in sys.path:
    sys.path.append(current_dir)
from simulation import *
input_path = '/scratchdata1/GeneralStorage/Zach/Designs/Sync/params_fig_Probe Number Tradeoff (18 Bits)_decoder_n_lyr_1_n_probes_50000_n_bit_18_replicate_1'
design_name = input_path.split('/')[-1]
data_path = '/scratchdata1/ExternalData'
paths = {}
paths['WeightMat'] = f"{input_path}/results/E_constrained.csv"
paths['Design'] = f"{data_path}/Allen_Cortex_Hippocampus_SmartSeq_2023Sep07"
paths['Reference'] = f"{data_path}/Allen_WMB_2024Mar06"
paths['Simulation'] = f"{data_path}/Zhaung_WMB/"
paths['Save'] = f"{input_path}/Simulation"
paths['Output'] = {}
for path in ['Design','Reference','Simulation','Results']:
    paths['Output'][path] = os.path.join(paths['Save'],path)
WeightMat = pd.read_csv(paths['WeightMat'], index_col=0)

Current working directory: /home/zach/PythonRepos/Design/Design/Results


In [3]:
section_acq_name = 'WellB-Section1'
adata = anndata.read_h5ad(f"/scratchdata1/Images2025/Haley/CIPHER/RNA_WTF4S7R3.B_2025Sep08/Processing_2025Sep11/{section_acq_name}/anndata/anndata.h5ad")
registration_path = f"/scratchdata1/Images2025/Haley/CIPHER/RNA_WTF4S7R3.B_2025Sep08/Registration_2025Sep12"
XYZC  = Registration_Class(adata.copy(),registration_path,section_acq_name,verbose=False,regularize=True).run()
adata.obs['ccf_x'] = XYZC['ccf_x']
adata.obs['ccf_y'] = XYZC['ccf_y']
adata.obs['ccf_z'] = XYZC['ccf_z']
XY = np.array(adata.obs[["ccf_z","ccf_y"]])
adata.obs['in_large_comp'] = geomu.in_graph_large_connected_components(XY,Section = None,max_dist = 0.05,large_comp_def = 0.1,plot_comp = False)
adata = adata[adata.obs['in_large_comp']==True].copy()

print(f"Keeping {adata.shape[0]} cells after component filtering for {section_acq_name}")

if adata.shape[0]<100:
    print(f" Not Enough Cells Found {adata.shape[0]} cells")
    print(f"Look Into Section {section_acq_name}")
    

n_cells_pre_nuc_filtering = adata.shape[0]
adata.layers['nuc_mask'] = basicu.filter_cells_nuc(adata)
adata = adata[np.sum(adata.layers['nuc_mask']==False,axis=1)<2].copy()
adata = adata[np.clip(np.array(adata.layers['raw']).copy().sum(1),1,None)>100].copy()

if adata.shape[0]<100:
    print(f" Not Enough Cells Found {adata.shape[0]} cells")
    print(f"Look Into Section {section_acq_name}")
    

print(f"Keeping {adata.shape[0]} cells after nuc filtering for {section_acq_name}")
n_cells_post_nuc_filtering = adata.shape[0]
if n_cells_post_nuc_filtering < 0.6 * n_cells_pre_nuc_filtering:
    print(f"Likely Gel issues: More than 40% cells Removed {adata.shape[0]} cells")
    print(f"Look Into Section {section_acq_name}")

if adata.shape[0]<60000:
    print(f" Not Enough Cells Found {adata.shape[0]} cells")
    print(f"Look Into Section {section_acq_name}")


adata.X = adata.layers['raw'].copy()

if adata.shape[0]<100:
    print(f" Not Enough Cells Found {adata.shape[0]} cells")
    print(f"Look Into Section {section_acq_name}")
    # continue

adata.X = basicu.normalize_fishdata_robust_regression(adata.X.copy())
# adata.X = adata.X.sum(1).mean()*adata.X/adata.X.sum(1)[:,None]


adata.X = basicu.image_coordinate_correction(adata.X.copy(),np.array(adata.obs[["image_x","image_y"]]))

# Problematic causes over correction
# if register_to_ccf: 
#     adata.X = basicu.correct_linear_staining_patterns(adata.X.copy(),np.array(adata.obs[["ccf_z","ccf_y"]]))
# else:
#     adata.X = basicu.correct_linear_staining_patterns(adata.X.copy(),np.array(adata.obs[["stage_x","stage_y"]]))

adata.layers['normalized'] = adata.X.copy()

adata.layers['classification_space'] = basicu.robust_zscore(adata.X.copy())

adata

Keeping 105159 cells after component filtering for WellB-Section1
Keeping 102015 cells after nuc filtering for WellB-Section1


AnnData object with n_obs × n_vars = 102015 × 18
    obs: 'label', 'stage_x', 'stage_y', 'image_x', 'image_y', 'image_idx', 'size', 'section_index', 'dapi', 'sum', 'dataset', 'well', 'ccf_x', 'ccf_y', 'ccf_z', 'in_large_comp'
    var: 'readout', 'hybe', 'channel'
    layers: 'nuc_processed_vectors', 'nuc_raw', 'processed_vectors', 'raw', 'nuc_mask', 'normalized', 'classification_space'

In [4]:
complete_reference = anndata.read_h5ad(os.path.join(paths['Output']['Reference'],f"{design_name}.h5ad"))
complete_reference.var

""
readout0
readout1
readout2
readout3
readout4
readout5
readout6
readout7
readout8
readout9


In [5]:
import warnings

# Use a regular expression to match the specific warning message
warnings.filterwarnings(
    "ignore",
    message="Observation names are not unique.*",
    category=UserWarning,
    module="anndata._core.anndata"
)

In [6]:
model_type = 'MLP'
likelihood_only = False
base_fname = f"{section_acq_name}_{model_type}_{'likelihood_only' if likelihood_only else 'full'}_decoded_anndata.h5ad"
print(base_fname)
fname = f"/scratchdata1/Images2025/Haley/CIPHER/RNA_WTF4S7R3.B_2025Sep08/Processing_2025Sep11/{section_acq_name}/anndata/{base_fname}"

adata_copy = adata.copy()
adata_copy.var = complete_reference.var.copy()
self = SingleCellAlignmentLeveragingExpectations(adata_copy,complete_reference=complete_reference,visualize=False,verbose=True)
self.n_models = 1
self.likelihood_only = likelihood_only
self.calculate_spatial_priors()
self.load_reference()
if model_type == 'LR':
    self.model = LogisticRegression(max_iter=1000,random_state=42)
elif model_type == 'MLP':
    self.model = MLPClassifier(hidden_layer_sizes=(WeightMat.shape[1]*3,), activation='relu', solver='adam', max_iter=1000, random_state=42)
else:
    raise ValueError(f"Invalid model type: {model_type}")
self.supervised_neuron_annotation()
self.supervised_harmonization()
self.determine_neighbors() # For Imputation
adata_copy = self.measured.copy()
adata_copy.write(fname)
plt.figure(figsize=[5,5])
plt.title(base_fname)
plt.axis('off')
plt.scatter(adata_copy.obs['ccf_z'],adata_copy.obs['ccf_y'],c=adata_copy.obs['subclass_color'],s=2.5,marker='.',linewidths=0, edgecolors=None)
plt.show()

WellB-Section1_MLP_full_decoded_anndata.h5ad


Harmonizing:  30%|███       | 111/369 [14:33<07:01,  1.64s/it] /home/zach/miniconda3/envs/dredfish_3.9/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.

Harmonizing:  33%|███▎      | 123/369 [15:28<18:28,  4.51s/it]/home/zach/miniconda3/envs/dredfish_3.9/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.

Harmonizing:  70%|██████▉   | 258/369 [20:11<01:45,  1.05it/s]/home/zach/miniconda3/envs/dredfish_3.9/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.

Harmonizing: 100%|██████████| 369/369 [21:42<00:00,  3.53s/it]


Mon Dec  8 18:12:27 2025 Building RP forest with 10 trees
Mon Dec  8 18:12:28 2025 NN descent for 11 iterations
	 1  /  11
	 2  /  11
	 3  /  11
	 4  /  11
	 5  /  11
	Stopping threshold met -- exiting after 5 iterations
Mon Dec  8 18:18:23 2025 Worst tree score: 0.71200000
Mon Dec  8 18:18:23 2025 Mean tree score: 0.73886316
Mon Dec  8 18:18:23 2025 Best tree score: 0.76084211
Mon Dec  8 18:18:28 2025 Forward diversification reduced edges from 35625 to 14951
Mon Dec  8 18:18:34 2025 Reverse diversification reduced edges from 14951 to 14951
Mon Dec  8 18:18:38 2025 Degree pruning reduced edges from 16392 to 16392
Mon Dec  8 18:18:38 2025 Resorting data and graph based on tree order
Mon Dec  8 18:18:38 2025 Building and compiling search function


AttributeError: 'Series' object has no attribute 'obs'

In [7]:
# raise
for i in range(15):
    self.measured.obs[f"reference_neighbor_{i}"] = 'Unknown'
for ct in tqdm(np.unique(self.measured_labels)):
    m = self.measured.obs[self.ref_level]==ct
    ref_m = self.reference.obs[self.ref_level]==ct
    if np.sum(ref_m)>20:
        tree = NNDescent(self.reference.layers['classification_space'][ref_m,:], metric='euclidean', n_neighbors=15,n_trees=10,verbose=False,random_state=42)
        neighbors,distances = tree.query(self.measured.layers['harmonized_classification_space'][m,:],k=15)
    else:
        neighbors = np.random.randint(0,np.sum(ref_m),size=(np.sum(m),15))
    ref = self.reference[ref_m]
    for i in range(neighbors.shape[1]):
        self.measured.obs.loc[m,f"reference_neighbor_{i}"] = np.array(ref.obs.index[neighbors[:,i]])
# X = np.zeros_like(self.measured.layers['harmonized_classification_space'])
# for i in range(neighbors.shape[1]):
#     X = X+np.array(self.reference[self.measured.obs[f"reference_neighbor_{i}"].values].layers['raw'])
# X = X/neighbors.shape[1]
# self.measured.layers['imputed'] = X.copy()

adata_copy = self.measured.copy()
adata_copy.write(fname)

 93%|█████████▎| 158/169 [06:07<00:42,  3.84s/it]/home/zach/miniconda3/envs/dredfish_3.9/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.

100%|██████████| 169/169 [06:54<00:00,  2.45s/it]
... storing 'neuron' as categorical
... storing 'neuron_color' as categorical
... storing 'subclass' as categorical
... storing 'subclass_color' as categorical
... storing 'reference_neighbor_0' as categorical
... storing 'reference_neighbor_1' as categorical
... storing 'reference_neighbor_2' as categorical
... storing 'reference_neighbor_3' as categorical
... storing 'reference_neighbor_4' as categorical
... storing 'reference_neighbor_5' as categorical
... storing 'reference_neighbor_6' as categorical
... storing 'reference_neighbor_7' as categorical
... storing 'reference_neighbor_8' as categorical
... storing 'reference_neighbor_9' as categorical
... storing 'reference_nei

In [8]:
self.measured.obs

,label,stage_x,stage_y,image_x,image_y,image_idx,size,section_index,dapi,sum,...,reference_neighbor_5,reference_neighbor_6,reference_neighbor_7,reference_neighbor_8,reference_neighbor_9,reference_neighbor_10,reference_neighbor_11,reference_neighbor_12,reference_neighbor_13,reference_neighbor_14
RNA_WTF4S7R3.B_2025Sep08_WellB-Section1_Cell3.0,3.0,1834.0,7416.0,578.0,810.0,27.0,180.0,WellB-Section1,193.578491,26590.378906,...,ATCGTAGCATTGACAC-480_B04observations1observati...,TCGGGACCATGGATGG-058_D01observations5observati...,CGTAGCGTCGCATGGC-034_A01observations5observati...,GCCAAATTCGGACAAG-070_B01observations5observati...,TCGCGAGAGTCTTGCA-098_B01observations5observati...,TCCTTTCCAGCGAGTA-390_D02observations1observati...,GGATCTATCCCGTGAG-071.1_B01observations1observa...,CACACTCTCGAGGTAG-046_C01observations5observati...,CTTGATTTCTTGGTCC-515_A03observations1observati...,GCTGCTTTCACGATGT-035_A01observations5observati...
RNA_WTF4S7R3.B_2025Sep08_WellB-Section1_Cell4.0,4.0,1905.0,7404.0,566.0,881.0,27.0,117.0,WellB-Section1,135.685974,24260.523438,...,CGGACACTCTGTCTCG-052_B01observations5observati...,TGACAACCAGGGTATG-050_A01observations5observati...,TTTCACAGTACCCACG-480_A04observations8observati...,ACGTAACAGGTAAGTT-480_B04observations1observati...,CGATCGGGTCGGCATC-010_B01observations5observati...,CAATACGGTTGGGACA-477_A01observations1observati...,ACTGATGAGGTCATAA-447_B06observations1observati...,CTGCGGAAGATGTCGG-005_C01observations5observati...,CTTAACTTCGGGAGTA-100_A01observations5observati...,GTGATGTCAGCGACCT-477_B01observations1observati...
RNA_WTF4S7R3.B_2025Sep08_WellB-Section1_Cell7.0,7.0,1933.0,7683.0,845.0,909.0,27.0,155.0,WellB-Section1,177.806488,25401.558594,...,CAATGACAGGTATTGA-447_B06observations1observati...,TACGGTACACCGTTGG-046_A01observations1observati...,ACCGTTCGTGCTTATG-480_B04observations8observati...,CTCTAATCACTTCGAA-056_B01observations1observati...,TGCCCATAGTTAGGTA-003_D01observations1observati...,TTCTACATCTGCGACG-058_A01observations4observati...,GTCTCGTGTATATCCG-003_F01observations1observati...,GTGCATACAAAGTGCG-053_B01observations1observati...,TACCTATTCGTTTGCC-1.16_B01observations1observat...,GAACATCTCTCGCATC-056_B01observations1observati...
RNA_WTF4S7R3.B_2025Sep08_WellB-Section1_Cell15.0,15.0,1825.0,7777.0,939.0,801.0,27.0,110.0,WellB-Section1,167.107086,27521.583984,...,CGCTATCAGCGATGAC-022_A01observations5observati...,AATCGACCAATAACGA-443_A03observations1observati...,CCTTGTGAGCTGAAGC-075.1_A01observations1observa...,GCGAGAAGTGAATATG-452_A01observations1observati...,TCGGGACAGGAATGGA-047_A01observations5observati...,CTAAGACTCGTTTATC-069_C01observations5observati...,GAATAAGGTTTAAGCC-031_A01observations5observati...,TATCAGGAGTCGCCGT-056_A01observations5observati...,CAAGCTAGTTGCTTGA-117_D01observations1observati...,CTTGAGAAGTGGTTAA-452_A01observations1observati...
RNA_WTF4S7R3.B_2025Sep08_WellB-Section1_Cell22.0,22.0,2036.0,7640.0,796.0,1012.0,27.0,237.0,WellB-Section1,135.760590,19220.195312,...,CACCACTTCTTGTCAT-034_A02observations1observati...,CTCAGAAGTCTAAAGA-1.16_A01observations1observat...,GCCAGTGTCTCAACCC-117_D01observations1observati...,GTCACGGGTGGCAAAC-1.04_E01observations1observat...,GACGGCTGTCTGCAAT-046_B01observations1observati...,GGATTACCACTTAACG-1.05_C01observations1observat...,GTGCGGTTCCAAACAC-098_A01observations1observati...,TAGAGCTCACTCTGTC-1.16_A01observations1observat...,TGCTACCAGGAACTGC-003_E01observations1observati...,ATCACGACAACACCTA-002_A01observations1observati...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RNA_WTF4S7R3.B_2025Sep08_WellB-Section1_Cell109246.0,109246.0,12109.0,8278.0,277.0,384.0,90.0,76.0,WellB-Section1,231.129059,5912.386719,...,AGGTCATGTATTAAGG-398_D01observations5observati...,AGGGAGTCAACGAGGT-344_C04observations11observat...,TAGGGTTTCGCGTCGA-390_B02observations1observati...,TCAGCCTAGTCTACCA-490_A06observations0observati...,CTGTGAAGTCACTTCC-452_B01observations1observati...,AGATGAAAGCCAGTAG-308_A03observations11observat...,ACACTGATCG

In [9]:
" Impute Gene Expression"
imputation_cells = np.unique(adata_copy.obs[[i for i in adata_copy.obs.columns if 'neighbor' in i]].values)
imputation_cells = np.unique([i.split('observation')[0] for i in imputation_cells])
imputation_cells.shape

(274683,)

In [10]:
imputation_cells[0:10]

array(['AAACCCAAGAAACACT-142_A01', 'AAACCCAAGAAACTCA-426_B04',
       'AAACCCAAGAACGTGC-407_B02', 'AAACCCAAGAAGATCT-232_A01',
       'AAACCCAAGAAGGCTC-213.2_B01', 'AAACCCAAGAAGTCAT-144_A01',
       'AAACCCAAGAAGTCAT-341_B01', 'AAACCCAAGAATCCCT-574_A02',
       'AAACCCAAGAATCTAG-198.1_A01', 'AAACCCAAGACCTTTG-194_A01'],
      dtype='<U26')

In [11]:
import gc
imputation_reference_adata = []
# V3
data_path = '/bluedata/ExternalData/Allen_WMB_2024Mar06/expression_matrices/WMB-10Xv3/20230630'
for file in tqdm(os.listdir(data_path)):
    if 'raw.h5ad' in file:
        temp_adata = anndata.read_h5ad(os.path.join(data_path,file),backed='r')
        shared_cells = np.intersect1d(imputation_cells,temp_adata.obs_names)
        print(f"Found {len(shared_cells)} cells in {file}")
        imputation_reference_adata.append(temp_adata[shared_cells])
        del temp_adata
        gc.collect()
#V2
data_path = '/bluedata/ExternalData/Allen_WMB_2024Mar06/expression_matrices/WMB-10Xv2/20230630/'
for file in tqdm(os.listdir(data_path)):
    if 'raw.h5ad' in file:
        temp_adata = anndata.read_h5ad(os.path.join(data_path,file),backed='r')
        shared_cells = np.intersect1d(imputation_cells,temp_adata.obs_names)
        print(f"Found {len(shared_cells)} cells in {file}")
        imputation_reference_adata.append(temp_adata[shared_cells])
        del temp_adata
        gc.collect()
imputation_reference_adata = anndata.concat(imputation_reference_adata)
gc.collect()
imputation_reference_adata.write_h5ad(f"/scratchdata1/Images2025/Haley/CIPHER/RNA_WTF4S7R3.B_2025Sep08/Processing_2025Sep11/{section_acq_name}/anndata/imputation_reference_adata.h5ad")
imputation_reference_adata

  0%|          | 0/27 [00:00<?, ?it/s]

Found 14456 cells in WMB-10Xv3-MY-raw.h5ad


  4%|▎         | 1/27 [00:04<01:52,  4.31s/it]

Found 4630 cells in WMB-10Xv3-OLF-raw.h5ad


 11%|█         | 3/27 [00:06<00:46,  1.94s/it]

...

Found 9554 cells in WMB-10Xv2-OLF-raw.h5ad


 86%|████████▌ | 18/21 [00:27<00:04,  1.49s/it]

Found 5437 cells in WMB-10Xv2-HY-raw.h5ad


100%|██████████| 21/21 [00:29<00:00,  1.40s/it]
... storing 'cell_barcode' as categorical
... storing 'library_label' as categorical
... storing 'anatomical_division_label' as categorical


AnnData object with n_obs × n_vars = 274683 × 32285
    obs: 'cell_barcode', 'library_label', 'anatomical_division_label'

In [12]:
imputation_reference_adata

AnnData object with n_obs × n_vars = 274683 × 32285
    obs: 'cell_barcode', 'library_label', 'anatomical_division_label'

In [13]:
del self
del adata
gc.collect()

176

In [14]:
neighbor_columns = [i for i in adata_copy.obs.columns if 'neighbor' in i]
imputed = None
for i,column in tqdm(enumerate(neighbor_columns)):
    neighbors = [i.split('obs')[0] for i in adata_copy.obs[column]]
    neighbor = imputation_reference_adata[neighbors,:].X.toarray().astype(np.float32)
    if isinstance(imputed,type(None)):
        imputed = neighbor
    else:
        imputed = imputed + neighbor
    del neighbor
    gc.collect()
imputed = imputed/len(neighbor_columns)
# imputed_adata = anndata.AnnData(X=imputed,obs=adata.obs,var=pd.DataFrame(index=imputation_reference.var.index))
imputed_adata = anndata.AnnData(X=imputed,obs=adata_copy.obs,var=imputation_reference_adata.var)
fname = f"/scratchdata1/Images2025/Haley/CIPHER/RNA_WTF4S7R3.B_2025Sep08/Processing_2025Sep11/{section_acq_name}/anndata/{base_fname.replace('.h5ad','_imputed.h5ad')}"
imputed_adata.write(fname)

15it [45:55, 183.69s/it]
